<a href="https://colab.research.google.com/github/WilliamMejiaG/Packed_Bed_Membrane_Reactor_Simulation/blob/main/Packed_Bed_Reactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy import optimize
import pandas as pd

In [ ]:
def F_PBR(VF,D_i,D_o,D_shell,FACT,rho_B,i,j,k,l,m):
    ####################################################################
    #Inlet molar fluxes
    R=8.314472 #J/mol K
    Ar = (np.pi*(D_i**2))/4 #m2
    Dm = (D_o-D_i)/np.log(D_o/D_i)
    Pm = np.pi*Dm
    #VSweep = l*VF
    VCO2_0 = VF/(m+1)
    VH2_0 = m*VCO2_0
    nH2_0 = VH2_0/(22400*60) #V in cm3 STP/min
    nCO2_0 = VCO2_0/(22400*60) #V in cm3 STP/min
    #nSweep = VSweep/(22400*60) #V in cm3 STP/min
    T = j+273
    
    """
    PACKED BED MEMBRANE REACTOR MODEL
    """
    # ODE definition
    def F(y,X):
        F = np.zeros(5)
        """
        RETENTATE SIDE
        """
        #Molar fluxes retentate
    
        n_H2r = y[0] 
        n_CO2r = y[1] 
        n_MeOHr = y[2]
        n_COr = y[3]
        n_Wr = y[4]
   
        n_Tr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
    
        TERM = (i/(n_Tr)) 
    
        pH2 = TERM*n_H2r
        pCO2 = TERM*n_CO2r
        pMeOH = TERM*n_MeOHr
        pCO = TERM*n_COr
        pW = TERM*n_Wr
        
        """
        PERMEATE SIDE
        
        
        Perm_W = 0 #mol/ m2 s bar
        Perm_MeOH =  0 #mol/ m2 s bar
        Perm_H2 = 0 #mol/ m2 s bar
        Perm_CO2 = 0
        Perm_CO = 0
        
        #Molar fluxes permeate

        n_H2p = y[5] 
        n_CO2p = y[6] 
        n_MeOHp = y[7]
        n_COp = y[8]
        n_Wp = y[9]
        n_swp = y[10]
        n_Tp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
    
        Pp = FACT*i#bar
    
        #Molar fractions Permeate
        y_H2p = n_H2p/n_Tp
        y_CO2p = n_CO2p/n_Tp
        y_MeOHp = n_MeOHp/n_Tp
        y_COp = n_COp/n_Tp
        y_Wp = n_Wp/n_Tp
   
    
        #Partial pressures permeate

        pH2_p = y_H2p*Pp #bar
        pCO2_p = y_CO2p*Pp #bar
        pMeOH_p = y_MeOHp*Pp  #bar
        pCO_p = y_COp*Pp #bar
        pW_p = y_Wp*Pp  #bar
        """
        """
        KINETICS
        """
        #Equilibrium constants
        K1 = 10**((3066/T)-10.592)
        K3 = 10**((-2073.00/T)+2.029)
        #Parameters
    
        A = 1.07*np.exp(36696.00/(R*T)) #N0 mol/ kg cat s bar^2, Q J/mol
        C = 1.22e10*np.exp(-94765.00/(R*T)) # N0 mol/ kg cat s bar, Q J/mol
        D = 3453.38
        E = 0.499*np.exp(17197.00/(R*T)) # N0 bar^-0.5, Q J/mol
        N = 6.62e-11*np.exp(124119.00/(R*T)) # N0 bar^-1, Q J/mol
    
        #Reaction Equations
        B = 1 + D*(pW/pH2) + E*np.sqrt(pH2) + N*pW
        r1 = (A*pCO2*pH2*(1-((pW*pMeOH)/(K1*(pH2**3)*pCO2))))/(B**3)
        r3 = (C*pCO2*(1-((pW*pCO)/(K3*pH2*pCO2))))/B
    
        """
        Differential equations retentate
        """
        F[0] = Ar*(-3*r1-r3)*(rho_B)
        F[1] = Ar*(-r1-r3)*(rho_B)
        F[2] = Ar*(r1)*(rho_B)
        F[3] = (r3)*Ar*(rho_B)
        F[4] = (r1+r3)*Ar*(rho_B)
    
        """
        
        Differential equations permeate co-current mode
        
        F[5] = Pm*Perm_H2*(pH2-pH2_p)
        F[6] = Pm*Perm_CO2*(pCO2-pCO2_p)
        F[7] = Pm*Perm_MeOH*(pMeOH-pMeOH_p)
        F[8] = Pm*Perm_CO*(pCO-pCO_p)
        F[9] = Pm*Perm_W*(pW-pW_p)
        F[10] = 0
        """
        return F
    # Initial conditions
    
    y = np.array([nH2_0,nCO2_0,0.00,0.00,0.00])
    
    xStart = 0.0
    h = 1.0e-5
    X = np.arange(xStart,k,h,dtype=float)
    Y = odeint(F,y,X)
    
    #FOR FROMENT CO-CURRENT FUNCTION
    n_H2r = Y[:,0]
    n_CO2r = Y[:,1]
    n_MeOHr = Y[:,2]
    n_COr = Y[:,3]
    n_Wr = Y[:,4]
    """
    n_H2p = Y[:,5]
    n_CO2p = Y[:,6]
    n_MeOHp = Y[:,7]
    n_COp = Y[:,8]
    n_Wp = Y[:,9]
    n_swp = Y[:,10]
    """
    n_Tr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
    
    #n_Tp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
    
    pH2r = (n_H2r/n_Tr)*i
    pCO2r = (n_CO2r/n_Tr)*i
    pMeOHr = (n_MeOHr/n_Tr)*i
    pCOr = (n_COr/n_Tr)*i
    pWr = (n_Wr/n_Tr)*i
    
    """
    pH2p = (n_H2p/n_Tp)*i
    pCO2p = (n_CO2p/n_Tp)*i
    pMeOHp = (n_MeOHp/n_Tp)*i
    pCOp = (n_COp/n_Tp)*i
    pWp = (n_Wp/n_Tp)*i
    pswp = (n_swp/n_Tp)*i
    """
              
    #Graphic for conversion
        
    X_CO2 = ((nCO2_0-n_CO2r)/nCO2_0)*100      
    S_MeOH = ((n_MeOHr)/(nCO2_0-n_CO2r))*100
    Y_MeOH = (X_CO2*S_MeOH)/100
    
    
    
    
    FROMENT = "FROMENT"
    PACKED_BED_REACTOR = "PACKED_BED_REACTOR"
         
    return X,X_CO2,S_MeOH,Y_MeOH,n_H2r,n_CO2r,n_MeOHr,n_COr,n_Wr,pH2r,pCO2r,pMeOHr,pCOr,pWr,PACKED_BED_REACTOR,FROMENT

In [ ]:
i = 10
j = 250
k = 0.10
VF = 250
l = 0
m = 3
D_i = 0.0067 #m
D_o = 0.01 #m
D_shell = 0.02#m
FACT = 1
rho_B= 1142 #kg cat/m3 lecho


In [ ]:
X,X_CO2,S_MeOH,Y_MeOH,n_H2r,n_CO2r,n_MeOHr,n_COr,n_Wr,pH2r,pCO2r,pMeOHr,pCOr,pWr,PACKED_BED_REACTOR,FROMENT= F_PBR(VF,D_i,D_o,D_shell,FACT,rho_B,i,j,k,l,m)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:166: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
data = np.array([X,X_CO2,S_MeOH,Y_MeOH,n_H2r,n_CO2r,n_MeOHr,n_COr,n_Wr,pH2r,pCO2r,pMeOHr,pCOr,pWr])
data_transposed = data.transpose()
df_PBR = pd.DataFrame(data_transposed,columns=["Reactor_Length","Conversion",'Selectivity','Yield','n_H2r','n_CO2r','n_MeOHr','n_COr','n_Wr','pH2r','pCO2r','pMeOHr','pCOr','pWr'])
df_PBR

,Reactor_Length,Conversion,Selectivity,Yield,n_H2r,n_CO2r,n_MeOHr,n_COr,n_Wr,pH2r,pCO2r,pMeOHr,pCOr,pWr
0,0.00000,0.000000,NaN,NaN,0.000140,0.000047,0.000000e+00,0.000000e+00,0.000000e+00,7.500000,2.500000,0.000000,0.000000,0.000000
1,0.00001,0.315060,61.487029,0.193721,0.000139,0.000046,9.008615e-08,5.642630e-08,1.465124e-07,7.489692,2.494540,0.004848,0.003036,0.007884
2,0.00002,0.589310,60.070748,0.354003,0.000139,0.000046,1.646219e-07,1.094248e-07,2.740466e-07,7.480808,2.489674,0.008866,0.005893,0.014759
3,0.00003,0.834113,58.822443,0.490645,0.000139,0.000046,2.281647e-07,1.597225e-07,3.878872e-07,7.472948,2.485244,0.012296,0.008608,0.020904
4,0.00004,1.056471,57.702012,0.609605,0.000138,0.000046,2.834844e-07,2.078059e-07,4.912904e-07,7.465864,2.481151,0.015287,0.011206,0.026493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.09995,17.082101,7.386382,1.261749,0.000130,0.000039,5.867510e-07,7.356934e-06,7.943685e-06,7.054364,2.086108,0.031744,0.398020,0.429764
9996,0.09996,17.082101,7.386382,1.261749,0.000130,0.000039,5.867510e-07,7.356934e-06,7.943685e-06,7.054364,2.086108,0.031744,0.398020,0.429764
9997,0.09997,17.082101,7.386382,1.261749,0.000130,0.000039,5.867510e-07,7.356934e-06,7.943685e-06,7.054364,2.086108,0.031744,0.398020,0.429764
9998,0.09998,17.082101,7.386382,1.261749,0.000130,0.000039,5.867510e-07,7.356934e-06,7.943685e-06,7.054364,2.086108,0.031744,0.398020,0.429764


In [ ]:
#To export your data frame to Excel
df_PBR.to_excel('PBR_10_250.xlsx',index=False,header=False) 
from google.colab import files
files.download('PBR_10_250.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
plt.plot(X,X_CO2);
print(X_CO2[-1])
print(S_MeOH[-1])
print(Y_MeOH[-1])

In [ ]:
plt.plot(X,pH2r, color = '#1f77b4');
plt.plot(X,pH2p, color = '#ff7f0e');

In [ ]:
plt.plot(X,n_H2r, color = '#1f77b4');
plt.plot(X,n_H2p, color = '#ff7f0e');

In [ ]:
plt.plot(X,pCO2r);
plt.plot(X,pCO2p);

In [ ]:
plt.plot(X,n_CO2r, color = '#1f77b4');
plt.plot(X,n_CO2p, color = '#ff7f0e');

In [ ]:
plt.plot(X,pCOr);
plt.plot(X,pCOp);

In [ ]:
plt.plot(X,n_COr);
plt.plot(X,n_COp);

In [ ]:
plt.plot(X,pWr);
plt.plot(X,pWp);

In [ ]:
plt.plot(X,n_Wr);
plt.plot(X,n_Wp);

In [ ]:
plt.plot(X,pMeOHr);
plt.plot(X,pMeOHp);

In [ ]:
plt.plot(X,n_MeOHr);
plt.plot(X,n_MeOHp);

In [ ]:
nTr = n_H2r+n_CO2r+n_MeOHr+n_COr+n_Wr
nTp = n_H2p+n_CO2p+n_MeOHp+n_COp+n_Wp+n_swp
plt.plot(X,nTr, color='blue');
plt.plot(X,nTp, color='red');

print(nTp[0])
print(nTr[0])
print(nTp[-1])
print(nTr[-1])